In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
def get_canonical_3d(world_3d, C):
    num_frames = len(data['world_3d'])
    canonical_3d = world_3d.copy()
    cam_origin = C.copy()
    pelvis = world_3d[:, 0].copy()
    vec_cam_origin_to_pelvis = pelvis - cam_origin
    mag_cam_origin_to_pelvis = np.expand_dims(np.linalg.norm(vec_cam_origin_to_pelvis, axis=1), axis=1).repeat(3, axis=1) # (F, 3)
    vec_cam_forward = np.multiply(np.expand_dims(R[2], 0).repeat(num_frames, axis=0),  mag_cam_origin_to_pelvis)
    canonical_pelvis = cam_origin + vec_cam_forward
    canonical_3d = canonical_3d - np.expand_dims(pelvis, 1) + np.expand_dims(canonical_pelvis, 1)
    return canonical_3d

In [3]:
h36m_3d_world, h36m_cam_param = load_h36m()

==> Loading 3D data wrt World CS...


### canonical_3d, joint_2d_from_canonical_3d

In [ ]:
save_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl'
if not os.path.exists(save_folder): os.makedirs(save_folder)

In [10]:
for item in tqdm(glob(save_folder + '/*.pkl')):
    data = readpkl(item)
    if ('canonical_3d' in data) and ('joint_2d_from_canonical_3d' in data): continue
    cam_param = data['cam_param'][0].reshape(1)[0].copy()
    int_mat = cam_param['intrinsic']
    ext_mat = cam_param['extrinsic']
    cam_proj = int_mat @ ext_mat
    R = ext_mat[:3, :3]
    t = ext_mat[:3, 3]
    C = T_to_C(R, t)
    world_3d = np.array(data['world_3d']).copy() # (F, J, 3)
    
    canonical_3d = get_canonical_3d(world_3d, C)
    pose_2d_from_3d_canonical = projection(canonical_3d, cam_proj)[..., :2]
    
    data['canonical_3d'] = canonical_3d.tolist()
    data['joint_2d_from_canonical_3d'] = pose_2d_from_3d_canonical.tolist()
    
    savepkl(data, item)
    #break

  1%|▏         | 12/836 [00:02<02:05,  6.57it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x775c983c70a0>>
Traceback (most recent call last):
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  8%|▊         | 68/836 [00:13<02:14,  5.70it/s]

### 2.5d_factor for canonical_3d

In [4]:
load_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl'
save_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl_canonical_3d_same_dist'
if not os.path.exists(save_folder): os.makedirs(save_folder)

In [5]:
subject_list = ['S8', 'S9', 'S11']

In [7]:
for item in glob(load_folder + '/*.pkl'):
    subject = glob(load_folder + '/*.pkl')[0].split('/')[-1].split('_')[0]
    if subject not in subject_list: continue
    save_path = item.replace('h36m_gt_pkl', 'h36m_gt_pkl_canonical_3d_same_dist')
    if os.path.exists(save_path): 
        print('already exists', save_path)
        continue
    data = readpkl(item)
    
    cam_param = data['cam_param'][0].reshape(1)[0].copy()
    int_mat = cam_param['intrinsic']
    ext_mat = cam_param['extrinsic']
    cam_proj = int_mat @ ext_mat
    R = ext_mat[:3, :3]
    t = ext_mat[:3, 3]
    C = T_to_C(R, t)
    num_frames = len(data['world_3d'])
    world_3d = np.array(data['world_3d']).copy() # (F, J, 3)
    canonical_3d = get_canonical_3d(world_3d, C)

    fx, fy, cx, cy = int_mat[0, 0], int_mat[1, 1], int_mat[0, 2], int_mat[1, 2]
    for frame_num in tqdm(range(num_frames)):
        # world to camera
        pos = canonical_3d[frame_num].copy()
        cam_3d = World2CameraCoordinate(pos, cam_param['extrinsic']) * 1000 # World coordinate -> Camera coordinate
        cam_3d_hat = get_rootrel_pose(cam_3d)
        
        # camera to image
        box = infer_box(cam_3d, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0)
        img_2d, img_3d = camera_to_image_frame(cam_3d, box, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0) 
        img_3d_hat = get_rootrel_pose(img_3d) # (17, 3) # root-relative pose  
        
        # 2.5d factor
        pred_lambda, losses = optimize_scaling_factor(img_3d_hat, cam_3d_hat, gpus='1') # x,y,z 사용
        
        # joint 2.5d image
        img_25d = img_3d * pred_lambda
    
        # store
        data['joint_2d'][frame_num] = np.array(img_2d).copy()
        data['joint3d_image'][frame_num] = np.array(img_3d).copy()
        data['joints_2.5d_image'][frame_num] = np.array(img_25d).copy()
        data['2.5d_factor'][frame_num] = np.array(pred_lambda).copy()
        #break
    savepkl(data, save_path)
    #break